In [18]:
import pandas as pd
import geopandas as gp
import numpy as np
import rasterstats
import os
import fiona
import rasterio
import rasterio.mask

In [2]:
output_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files"

Buildings processing
- create raster topo from contours
- subtract out contours from building heights to extrude in rhino

In [3]:
default_height = 20

In [4]:
# set extent boundary
extent_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/neighborhood_extent.shp"
extent = gp.read_file(extent_path)

In [5]:
# set watershed boundary
watershed_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/green_duwamish_watershed.shp"
watershed = gp.read_file(watershed_path)

In [6]:
# find intersection of watershed boundary and extent
watershed_extent_intersect = gp.overlay(watershed, extent, how='intersection')
watershed_extent_intersect.to_file(os.path.join(output_path, "watershed_boundary_intersect.shp"))

In [8]:
# roads processing
roads_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Projected/seattle_streets_projected.shp"
roads = gp.read_file(roads_path)
roads_clip = gp.overlay(roads, extent, how='intersection')
roads_clip.to_file(os.path.join(output_path, "neighborhood_streets.shp"))

In [9]:
buildings_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/buildings_projected_trim.shp"
buildings = gp.read_file(buildings_path)

In [10]:
# floodplain extent for clipping
floodplain_path = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Boundary/floodplain.shp"
floodplain = gp.read_file(floodplain_path)


In [11]:
buildings['centroid_geom'] = buildings.centroid
buildings.set_geometry('centroid_geom',inplace = True)
buildings_clip = gp.overlay(buildings, floodplain, how = "intersection")

In [13]:
buildings_merge = pd.merge(buildings_clip, buildings[['OBJECTID','geometry']], left_on ='OBJECTID', right_on = 'OBJECTID', how = 'left')
buildings_merge.set_geometry('geometry_y', inplace = True)

In [ ]:
# fill 0 and NA values to default_height
buildings_merge['BP99_APEX'].fillna(default_height, inplace = True)
buildings_merge['BP99_APEX'] = np.where(buildings_merge['BP99_APEX']==0,default_height, buildings_merge['BP99_APEX'])

In [ ]:
# TODO: additional processing to make resolution a bit better

# bounding box of each building to simplify geometries

# remove buildings that are < minimum area

In [ ]:
buildings_merge[['OBJECTID', 'BP99_TYPE', 'BP99_APEX','geometry_y']].to_file(os.path.join(output_path,"neighborhood_buildings.shp"))

TOPO PROCESSING
- merging bathymetry data with topography dem

In [31]:
# mask topo_rast by extent
topo_rast = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/topo.tif"

with fiona.open(extent_path, "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

with rasterio.open(topo_rast) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta

In [35]:
topo_outpath = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Neighborhood_Files/neighborhood_topo.asc"

# convert to ascii file
out_meta.update({"driver": "AAIGrid",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})

with rasterio.open(topo_outpath, "w", **out_meta) as dest:
    dest.write(out_image)

EXTRA PROCESSING FOR ASSIGNING ABSOLUTE BUILDING HEIGHTS FROM TERRAIN

In [ ]:
## used topo to raster tool with 5ft contours and 5 ft pixel resolution
# there may be better ways to get DEM / elevation values as raster
topo_rast = "/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/min_bath_topo.tif"
zonal_stats = rasterstats.zonal_stats(buildings_merge, topo_rast, stats="count min mean max median std")
stats_df = pd.DataFrame.from_dict(zonal_stats).reset_index()
buildings_merge.reset_index(inplace = True)
merged_heights = pd.merge(buildings_merge, stats_df, how = 'left', left_on = 'index', right_on = 'index')
merged_heights['height_upd'] = np.where((merged_heights['BP99_APEX'].isna()==False) | (merged_heights['mean'].isna()==False), default_height, (merged_heights['BP99_APEX']-merged_heights['mean']))
merged_heights.set_geometry('geometry_y')
merged_heights[['OBJECTID','BP99_APEX','mean','min','max','median','std','height_upd','geometry_y']].to_file("/home/nweiss/gdrive/Year 2/Summer - Duwamish/GIS/Processed/building_heights_upd.shp")